In [70]:
simport matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [71]:
#importing data, spray data for the test set (2008,2010,2012,2014) is not provided. Therefore, spray info is not used for the analysis
train = pd.read_csv('./assets/train.csv')
test =pd.read_csv('./assets/test.csv')
weather = pd.read_csv('./assets/weather.csv')
spray =pd.read_csv('./assets/spray.csv')
mapdata = np.loadtxt("./assets/mapdata_copyright_openstreetmap_contributors.txt")

In [72]:
from datetime import datetime
train['Date'] = pd.to_datetime(train['Date'], format='%Y-%m-%d')
test['Date'] = pd.to_datetime(test['Date'], format='%Y-%m-%d')
weather['Date'] = pd.to_datetime(weather['Date'], format='%Y-%m-%d')
spray['Date'] = pd.to_datetime(spray['Date'], format='%Y-%m-%d')

In [73]:
train['week'] = train['Date'].dt.weekofyear
train['year'] = train['Date'].dt.year
test['week'] = test['Date'].dt.weekofyear
test['year'] = test['Date'].dt.year
spray['year']=spray['Date'].dt.year

In [74]:
train.columns


Index([u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'NumMosquitos', u'WnvPresent', u'week', u'year'],
      dtype='object')

In [75]:
train = train.drop(train[[1,3,4,5,6,9,10]], axis=1)

In [76]:
train.head(1)

,Date,Species,Latitude,Longitude,WnvPresent,week,year
0,2007-05-29,CULEX PIPIENS/RESTUANS,41.95469,-87.800991,0,22,2007


In [77]:
test =test.drop(test[[0,2,4,5,6,7,10]], axis=1)

In [78]:
test.head()

,Date,Species,Latitude,Longitude,week,year
0,2008-06-11,CULEX PIPIENS/RESTUANS,41.95469,-87.800991,24,2008
1,2008-06-11,CULEX RESTUANS,41.95469,-87.800991,24,2008
2,2008-06-11,CULEX PIPIENS,41.95469,-87.800991,24,2008
3,2008-06-11,CULEX SALINARIUS,41.95469,-87.800991,24,2008
4,2008-06-11,CULEX TERRITANS,41.95469,-87.800991,24,2008


In [10]:
train = pd.concat([train , pd.get_dummies(train['Species'])], axis=1)  

In [11]:
test = pd.concat([test , pd.get_dummies(test['Species'])], axis=1)  

In [41]:
test.head()

,Date,Latitude,Longitude,week,year,CULEX ERRATICUS,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,UNSPECIFIED CULEX
0,2008-06-11,41.95469,-87.800991,24,2008,0,0,1,0,0,0,0,0
1,2008-06-11,41.95469,-87.800991,24,2008,0,0,0,1,0,0,0,0
2,2008-06-11,41.95469,-87.800991,24,2008,0,1,0,0,0,0,0,0
3,2008-06-11,41.95469,-87.800991,24,2008,0,0,0,0,1,0,0,0
4,2008-06-11,41.95469,-87.800991,24,2008,0,0,0,0,0,0,1,0


In [13]:
test =test.drop(test[[1]], axis=1)
train =train.drop(train[[1]], axis=1)

In [14]:
w30 = pd.read_csv('weather_ave_30.csv')

In [15]:
w30 = w30.drop('Datetime_Date', axis=1)


In [16]:
w30['Date'] = pd.to_datetime(w30['Date'], format='%Y-%m-%d')

In [17]:
# Adding the weather data to the train and test data
train_add =train.join(w30.set_index('Date'), on='Date')
test_add = test.join(w30.set_index('Date'), on='Date')

In [18]:
test_add['UNSPECIFIED CULEX'].value_counts()



0    101948
1     14345
Name: UNSPECIFIED CULEX, dtype: int64

In [19]:
train_add['UNSPECIFIED CULEX']= 0


In [20]:
y =train_add['WnvPresent']


In [21]:
train_add = train_add.drop('WnvPresent', axis=1)


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report,roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold ,train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV



/Users/khaterehmohajery1/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [23]:
train_add.columns

Index([u'Date', u'Latitude', u'Longitude', u'week', u'year',
       u'CULEX ERRATICUS', u'CULEX PIPIENS', u'CULEX PIPIENS/RESTUANS',
       u'CULEX RESTUANS', u'CULEX SALINARIUS', u'CULEX TARSALIS',
       u'CULEX TERRITANS', u'Tmax', u'Tmin', u'Tavg', u'Depart', u'DewPoint',
       u'WetBulb', u'Heat', u'Cool', u'PrecipTotal', u'StnPressure',
       u'SeaLevel', u'ResultSpeed', u'ResultDir', u'AvgSpeed', u'HZ', u'VC',
       u'FU', u'BC', u'SQ', u'FG+', u'MI', u'TS', u'DZ', u'RA', u'BR', u'FG',
       u'SN', u'UNSPECIFIED CULEX'],
      dtype='object')

In [24]:
X =train_add[[u'Latitude', u'Longitude', u'week', u'year',
       u'CULEX ERRATICUS', u'CULEX PIPIENS', u'CULEX PIPIENS/RESTUANS',
       u'CULEX RESTUANS', u'CULEX SALINARIUS', u'CULEX TARSALIS',
       u'CULEX TERRITANS', u'Tmax', u'Tmin', u'Tavg', u'Depart', u'DewPoint',
       u'WetBulb', u'Heat', u'Cool', u'PrecipTotal', u'StnPressure',
       u'SeaLevel', u'ResultSpeed', u'ResultDir', u'AvgSpeed', u'HZ', u'VC',
       u'FU', u'BC', u'SQ', u'FG+', u'MI', u'TS', u'DZ', u'RA', u'BR', u'FG',
       u'SN', u'UNSPECIFIED CULEX']]

In [42]:
test_add= test_add.drop('Date', axis=1)

In [29]:
scaler = StandardScaler()
X= scaler.fit_transform(X)


In [43]:
test_add =scaler.fit_transform(test_add)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [38]:
##Grid search over Random Forest parameters
# model evaluation function
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    probabilities = model.predict_proba(X_test)
    #cm = confusion_matrix(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))

    confusion = pd.DataFrame(conmat, index=['1', '0'],
                         columns=['predicted_1','predicted_0'])

    cr = classification_report(y_test, y_pred)
    
    print 'confusion matrix:'
    print confusion
    print 'classification_report:'
    print cr
    print 'Accuracy of the model on test:',a
    return probabilities
#params = {'max_features ': [0.5,1.0],'max_depth':[0.5,1.0],'n_estimators':[5,10]}
max_depths = [0.3, 0.5,1.0]
max_features = [0.3,0.5,1.0]
n_estimators = [5000]

rf = RandomForestClassifier(n_jobs=-1,random_state = 33, class_weight='balanced')
gsrf = GridSearchCV(estimator = rf,param_grid=dict(max_depth = max_depths, max_features=max_features,n_estimators=n_estimators), n_jobs=-1,cv=3,scoring='roc_auc')
gsrf.fit(X_train, y_train)
print 'best parameters for the model:',gsrf.best_params_
print 'best score on train:',gsrf.best_score_
probability = evaluate_model(gsrf.best_estimator_)


KeyboardInterrupt: 

In [46]:
##Grid search over Knn parameters
# model evaluation function
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    probabilities = model.predict_proba(X_test)
    #cm = confusion_matrix(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))

    confusion = pd.DataFrame(conmat, index=['1', '0'],
                         columns=['predicted_1','predicted_0'])

    cr = classification_report(y_test, y_pred)
    
    print 'confusion matrix:'
    print confusion
    print 'classification_report:'
    print cr
    print 'Accuracy of the model on test:',a
    return probabilities

n_neighbors =  range(5, 30)
knn = KNeighborsClassifier()
gsknn = GridSearchCV(estimator = knn,param_grid=dict( n_neighbors=n_neighbors), n_jobs=-1,cv=3,scoring='roc_auc')
gsknn.fit(X_train, y_train)
print 'best parameters for the model:',gsknn.best_params_
print 'best score on train:',gsknn.best_score_
probability = evaluate_model(gsknn.best_estimator_)


best parameters for the model: {'n_neighbors': 26}
best score on train: 0.781501809087
confusion matrix:
   predicted_1  predicted_0
1            0          182
0            0         3285
classification_report:
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      3285
          1       0.00      0.00      0.00       182

avg / total       0.90      0.95      0.92      3467

Accuracy of the model on test: 0.947505047592


In [52]:
predictions =gsknn.best_estimator_.predict(test_add)

In [67]:
predictions = pd.Series(predictions)
submit= pd.concat([test['Id'], predictions], axis= 1)


In [68]:
submit.columns=['Id','WnvPresent']

In [69]:
submit.to_csv('submission.csv',index_label=False)


In [45]:
##Grid search over Logestic Regression parameters
# model evaluation function
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
   
    a = accuracy_score(y_test, y_pred)
    probabilities = model.predict_proba(X_test)
#cm = confusion_matrix(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))

    confusion = pd.DataFrame(conmat, index=['1', '0'],
                        columns=['predicted_1','predicted_0'])

    cr = classification_report(y_test, y_pred)
   
    print 'confusion matrix:'
    print confusion
    print 'classification_report:'
    print cr
    print 'Accuracy of the model on test:',a
    return probabilities

penalty =  ['l2','l1']
C = [0.1,1.0,10]
lg = LogisticRegression()
gslg = GridSearchCV(estimator = lg,param_grid=dict(C=C, penalty=penalty), n_jobs=-1,cv=3,scoring='roc_auc')
gslg.fit(X_train, y_train)
print 'best parameters for the model:',gslg.best_params_
print 'best score on train:',gslg.best_score_
probability = evaluate_model(gslg.best_estimator_)

best parameters for the model: {'penalty': 'l1', 'C': 1.0}
best score on train: 0.835144536791
confusion matrix:
   predicted_1  predicted_0
1            0          182
0            1         3284
classification_report:
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      3285
          1       0.00      0.00      0.00       182

avg / total       0.90      0.95      0.92      3467

Accuracy of the model on test: 0.947216613787


In [44]:
##Grid search over Gradiant boosting
# model evaluation function
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    probabilities = model.predict_proba(X_test)
    #cm = confusion_matrix(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[1,0]))
    confusion = pd.DataFrame(conmat, index=['1', '0'],
                         columns=['predicted_1','predicted_0'])
    cr = classification_report(y_test, y_pred)
    
    print 'confusion matrix:'
    print confusion
    print 'classification_report:'
    print cr
    print 'Accuracy of the model on test:',a
    return probabilities
max_features = [0.1,.5,1.0]
max_depth = [0.1,0.5,1.0]
n_estimators = [3, 5, 10, 50]
gbc = GradientBoostingClassifier()
gsgbc = GridSearchCV(estimator = gbc,param_grid=dict(max_features=max_features, max_depth=max_depth, n_estimators=n_estimators), n_jobs=-1,cv=3, scoring = 'roc_auc')
gsgbc.fit(X_train, y_train)
print 'best parameters for the model:',gsgbc.best_params_
print 'best score on train:',gsgbc.best_score_
probability = evaluate_model(gsgbc.best_estimator_)

best parameters for the model: {'max_features': 0.5, 'n_estimators': 50, 'max_depth': 1.0}
best score on train: 0.815676865299
confusion matrix:
   predicted_1  predicted_0
1            0          182
0            0         3285
classification_report:
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      3285
          1       0.00      0.00      0.00       182

avg / total       0.90      0.95      0.92      3467

Accuracy of the model on test: 0.947505047592
